In [ ]:
import sys
from glob import glob
from os import path as osp
from datetime import datetime
from tqdm import tqdm
from skimage import io, transform
from matplotlib import pyplot as plt
import numpy as np
from math import sqrt
from statistics import mean
# torch imports
import torch
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader

# root path of project
from os import path as osp
import sys

# get root directory
import re
reg = '^.*/AquaPose'
project_root = re.findall(reg, osp.dirname(osp.abspath(sys.argv[0])))[0]
sys.path.append(project_root)

from lib.dataset.PoseDataset import PoseDataset
from lib.dataset.CycleDataset import CycleDataset

from lib.models.keypoint_rcnn import get_resnet50_pretrained_model

# utils
from lib.utils.slack_notifications import slack_message
from lib.utils.select_gpu import select_best_gpu
from lib.utils.rmsd import kabsch_rmsd, kabsch_rotate, kabsch_weighted_rmsd, centroid, centroid_weighted, rmsd, rmsd_weighted, kabsch

# references import
# source: https://github.com/pytorch/vision/tree/master/references/detection
from references.engine import train_one_epoch, evaluate
from references.utils import collate_fn

from references.transforms import RandomHorizontalFlip

from lib.matching.matching import *
from lib.utils.visual_utils import *

In [ ]:
device = select_best_gpu(min_mem=3000) if torch.cuda.is_available() else torch.device('cpu')
cpu = torch.device('cpu')
print(device)
print(cpu)

In [ ]:
# load dataset to get a set of poses to match to
ref_dataset = PoseDataset([osp.join(project_root,'data/vzf/freestyle/freestyle_1'), osp.join(project_root,'data/vzf/freestyle/freestyle_2'), osp.join(project_root,'data/vzf/freestyle/freestyle_3'), osp.join(project_root,'data/vzf/freestyle/freestyle_4')], train=False)

inference_dataset = PoseDataset([osp.join(project_root,'data/vzf/freestyle/freestyle_5')], train=False)
cycle_dataset = CycleDataset([osp.join(project_root,'data/vzf/freestyle/freestyle_5')])

In [ ]:
weight_dir = osp.join(project_root, 'weights')
weight_files = glob(osp.join(weight_dir,'*'))
model = get_resnet50_pretrained_model()
for i, f in enumerate(weight_files):
    print('{}, {}'.format(i,f))
model.load_state_dict(torch.load(weight_files[32], map_location=torch.device('cpu')))


In [ ]:
model.to(device)

In [ ]:
#anchor_ids = [40,43,45,47,48,50,52,54,57,58]
anchor_ids = [40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59]
anchor_dataset = torch.utils.data.Subset(ref_dataset, anchor_ids)

In [ ]:
transmat = build_transmat(len(anchor_ids))
flipped_transmat = build_transmat(2, [.95,.05])

In [ ]:
cycle_subs = torch.utils.data.Subset(cycle_dataset, [x for x in range(0,20)])

In [ ]:
model.eval()

In [ ]:



obs_lik_list, observations_list, flipped_list= get_observation_likelihood(model, inference_dataset, anchor_dataset, max_stride=3, device=device)

In [ ]:
phase_offset = [item_dict['offset'] for _, item_dict in cycle_subs]
for obs_lik, obs, flipped_mat in zip(obs_lik_list, observations_list, flipped_list):
    print('obs: {}'.format(obs))
    mls = viterbi_path(np.array([1.0/len(anchor_ids)]*len(anchor_ids)), transmat, obs_lik)

    # also get most likely sequence for being flipped or not
    flipped_observed = [flipped_mat[mls[i]][i] for i in range(0, len(mls))]
    obslik_flipped = [[ .75 - (.5 * flipped_observed[i]) for i in range(0,len(mls))]]
    obslik_flipped += [[ 1 - obslik_flipped[0][i] for i in range(0,len(mls))]]
    obslik_flipped = np.array(obslik_flipped)
    # print('obslik_flipped: {}'.format(obslik_flipped))

    mls_flipped = viterbi_path([.5,.5], flipped_transmat, obslik_flipped)
    print('mls_flipped: {}'.format(mls_flipped))

    print('{}'.format(mls))
    print(phase_offset)


    # for num, _ in enumerate(obs):
    #     print('obs {}: {}'.format(num, np.array(obs_lik)[:,num]))

    for obs_id, (img, anchor) in tqdm(enumerate(zip(range(0,len(cycle_subs)), mls))):
        
        inf_img, _ = cycle_subs[img]
        if device:
            inf_img = inf_img.to(device)
        warped_anchor = warp_anchor_on_pred(model, inf_img, anchor_dataset, anchor, True if mls_flipped[obs_id] else False, device=device)
        if device:
            del inf_img
        inf_img, _ = cycle_subs[img]
        plot_image_with_kps_skeleton(inf_img, [warped_anchor])

In [ ]:
torch._C._cuda_emptyCache()

In [ ]:
def get_observation_likelihood(model, inference_dataset, anchor_dataset, max_stride=1, device=None):
    model.eval()
    cpu = torch.device('cpu')
    
    observations_list = []
    obslik_list =[]
    flipped_list = []

    # initialize empty likelihood
    obslik = np.zeros((len(anchor_dataset), 0))


    last_id = 0

    while last_id < len(inference_dataset) - 1:
        # image ids (name of image.jpg), to allow for checking of stride
        prev_id = None
        cur_id = None

        observations = []
        hidden_states = []
        img_ids = []
        obslik = np.zeros((len(anchor_dataset), 0))
        flipped_mat = np.zeros((len(anchor_dataset),0))

        cur_range = range(last_id, len(inference_dataset))
        for id in tqdm(cur_range):
            img, target = inference_dataset[id]

            last_id = id

            if device:
                img = img.to(device)
            print(img)
            prediction = model([img])
            prediction = targets = [{k: v.cpu() for k, v in t.items()} for t in prediction]
            pred_box, pred_kps, pred_scores = get_max_prediction(prediction)

            if device:
                del img

            # print('pred_kps {}'.format(pred_kps))
            # print('pred_scores {}'.format(pred_scores))

            # get observed state
            best_ind, scores, flipped = get_most_similar_ind_and_scores(pred_kps, pred_scores, anchor_dataset, num=len(anchor_dataset),  filter_lr_confusion=False, occluded=False, translat_weights=T_WEIGHTS, kp_weights=KP_WEIGHTS)

            observations += [best_ind[0]]
            
            scores = np.array(scores)[np.argsort(best_ind)]
            flipped = np.array(flipped)[np.argsort(best_ind)]
            scores = np.power(scores, np.array([4.0] * len(scores)))
            if np.sum(scores) > 0:
                scores_norm = np.array(scores)/np.sum(np.array(scores))
            else:
                scores_norm = np.array([1/len(scores)] * len(scores))
            scores_norm = np.array([[score] for score in scores_norm])
            flipped = np.array([[f] for f in flipped])
            obslik = np.append(obslik, scores_norm, axis=1)
            flipped_mat = np.append(flipped_mat, flipped, axis=1)
            
            print(obslik)

        observations = np.array(observations)

        observations_list += [observations]
        obslik_list += [obslik]
        flipped_list += [flipped_mat]

    return np.array(obslik_list), np.array(observations_list), np.array(flipped_list)
